In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import getpass
import pdvega
import plotly.graph_objs as go

from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
from plotly.graph_objs import *

# for configuring connection 
from configobj import ConfigObj
import os

%matplotlib inline


import os


from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

#configure the notebook for use in offline mode
init_notebook_mode(connected=True)

In [ ]:
df= pd.read_csv("Analysis.csv")
df.head()

,Unnamed: 0,hospitalid,sodium,electivesurgery,vent,dialysis,gcs,urine,wbc,temperature,...,m11_True,m12_True,m13_True,m14_True,m15_True,m16_True,m17_True,m18_True,m19_True,m20_True
0,0,59.0,139.0,-1.0,0.0,0.0,15.0,-1.0,14.7,36.1,...,1,0,0,1,1,0,0,0,1,0
1,1,73.0,134.0,-1.0,0.0,0.0,13.0,-1.0,14.1,39.3,...,1,0,0,1,1,0,0,0,1,0
2,2,73.0,-1.0,1.0,1.0,0.0,15.0,-1.0,8.0,34.8,...,0,0,1,0,0,1,0,1,0,0
3,3,63.0,137.0,-1.0,0.0,0.0,15.0,-1.0,10.9,36.6,...,1,0,1,1,1,0,0,1,1,0
4,4,63.0,135.0,-1.0,0.0,0.0,15.0,-1.0,5.9,35.0,...,0,0,1,0,0,0,0,1,0,0


In [ ]:
del df['hospitalid']

df = df.drop(df.columns[[0]], axis=1)

In [ ]:
missing_values_count = df.isnull().sum()
missing_values_count

sodium                    0
electivesurgery           0
vent                      0
dialysis                  0
gcs                       0
urine                     0
wbc                       0
temperature               0
respiratoryrate           0
heartrate                 0
meanbp                    0
creatinine                0
ph                        0
hematocrit                0
albumin                   0
pao2                      0
pco2                      0
bun                       0
glucose                   0
bilirubin                 0
fio2                      0
age                       0
thrombolytics             0
aids                      0
hepaticfailure            0
lymphoma                  0
metastaticcancer          0
leukemia                  0
immunosuppression         0
cirrhosis                 0
                         ..
diaggroup_Neuro           0
diaggroup_Other           0
diaggroup_Overdose        0
diaggroup_PNA             0
diaggroup_RespMedOth

**We moved all the pre-processing including splitting>imputation>Standardization to the CV iterations**

In [ ]:
cols_to_norm=['gcs', 'urine', 'wbc', 'sodium',
       'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'creatinine',
       'ph', 'hematocrit', 'albumin', 'pao2', 'pco2', 'bun', 'glucose',
       'bilirubin', 'fio2', 'age', 'offset']

X=df.drop('destcopy', 1)
y=df['destcopy']
df_cols = list(X)     #fancy impute removes column names.

In [ ]:
df.columns

Index(['sodium', 'electivesurgery', 'vent', 'dialysis', 'gcs', 'urine', 'wbc',
       'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'creatinine',
       'ph', 'hematocrit', 'albumin', 'pao2', 'pco2', 'bun', 'glucose',
       'bilirubin', 'fio2', 'age', 'thrombolytics', 'aids', 'hepaticfailure',
       'lymphoma', 'metastaticcancer', 'leukemia', 'immunosuppression',
       'cirrhosis', 'readmit', 'offset', 'destcopy', 'admitsource_1.0',
       'admitsource_2.0', 'admitsource_3.0', 'admitsource_4.0',
       'admitsource_5.0', 'admitsource_6.0', 'admitsource_7.0',
       'admitsource_8.0', 'diaggroup_ARF', 'diaggroup_Asthma-Emphys',
       'diaggroup_CABG', 'diaggroup_CHF', 'diaggroup_CVA', 'diaggroup_CVOther',
       'diaggroup_CardiacArrest', 'diaggroup_ChestPainUnknown',
       'diaggroup_Coma', 'diaggroup_DKA', 'diaggroup_GIBleed',
       'diaggroup_GIObstruction', 'diaggroup_Neuro', 'diaggroup_Other',
       'diaggroup_Overdose', 'diaggroup_PNA', 'diaggroup_RespMedOther',


**Logistic Regression**

In [ ]:
from collections import Counter

In [ ]:
from sklearn.model_selection import KFold
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import f1_score
from imblearn.metrics import classification_report_imbalanced
from yellowbrick.classifier import ROCAUC
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
import xgboost as xgb
import lightgbm as lgbm
from collections import Counter
from sklearn.cluster import KMeans



classes=['Death','Home','Nursing Home','Rehabilitation']


kf = KFold(n_splits=10)

for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]  # See comment on ravel and  y_train
    
    n_clusters = len(np.unique(y_train))
    
#------------------------------Standardize Testing Set------------------------------------
    std_scale = preprocessing.StandardScaler().fit(X_train[cols_to_norm])
    X_train[cols_to_norm] = std_scale.transform(X_train[cols_to_norm])
    X_test[cols_to_norm] = std_scale.transform(X_test[cols_to_norm])
#------------------------------------------------------------------------------------------

 # Hyperparameters are optimized using hyperopt

       
    clf = KMeans(n_clusters = n_clusters, random_state=42)
    clf.fit(X_train[['vent', 'temperature', 'bun', 'fio2',   'gcs','age', 'offset','diaggroup_CVA', 'diaggroup_CardiacArrest', 'diaggroup_Sepsis']])
    y_labels_train = clf.labels_
    y_labels_test = clf.predict(X_test[['vent', 'temperature', 'bun', 'fio2',   'gcs','age', 'offset','diaggroup_CVA', 'diaggroup_CardiacArrest', 'diaggroup_Sepsis']])
    X_train['km_clust'] = y_labels_train
    print("hi")
    X_train.shape
    X_test['km_clust'] = y_labels_test
    X_test.shape
    
    
    
    
#------------------------------------------------------------------------------------------
    
    sm = SMOTENC(random_state=50, categorical_features=[1,2,3,22,23,24,25,26,27,28,29,30,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61, 62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81])
    X_train_oversampled, y_train_oversampled = sm.fit_sample(X_train, y_train)
    print(sorted(Counter(y_train).items()))
    model = linear_model.LogisticRegression(C= 2.5,max_iter= 100,tol= 6.75e-05,penalty='l1', class_weight='balanced')  
    model.fit(X_train_oversampled, y_train_oversampled)  
    y_pred = model.predict(X_test)
    visualizer = ROCAUC(model, classes=classes)
    visualizer.fit(X_train_oversampled, y_train_oversampled)  # Fit the training data to the visualizer
    visualizer.score(X_test, y_test)  # Evaluate the model on the test data
    visualizer.poof("LR_Mixed_Main_{}.pdf".format(fold), clear_figure=True) 
    print(f'For fold {fold}:')
    print(f'Accuracy: {model.score(X_test, y_test)}')
    f1=f1_score(y_test, y_pred, average='micro')
    print(f'f-score: {f1}')
    print(classification_report_imbalanced(y_test, y_pred))
    


    
    

C:\Users\llois\Anaconda\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\llois\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\llois\Anaconda\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

hi
[(1, 9194), (2, 59442), (3, 12662), (4, 4335)]


C:\Users\llois\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\llois\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

C:\Users\llois\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\llois\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



For fold 1:
Accuracy: 0.6286915396741987
f-score: 0.6286915396741987
                   pre       rec       spe        f1       geo       iba       sup

          1       0.36      0.53      0.93      0.43      0.70      0.47       642
          2       0.83      0.73      0.63      0.78      0.68      0.47      6776
          3       0.37      0.32      0.88      0.35      0.53      0.27      1716
          4       0.11      0.33      0.89      0.17      0.55      0.28       381

avg / total       0.69      0.63      0.71      0.65      0.65      0.42      9515



C:\Users\llois\Anaconda\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\llois\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\llois\Anaconda\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

...

C:\Users\llois\Anaconda\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\llois\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\llois\Anaconda\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

hi
[(1, 8679), (2, 59618), (3, 13222), (4, 4115)]


C:\Users\llois\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\llois\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.

C:\Users\llois\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\llois\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



For fold 10:
Accuracy: 0.6359049821315955
f-score: 0.6359049821315955
                   pre       rec       spe        f1       geo       iba       sup

          1       0.51      0.57      0.92      0.54      0.72      0.51      1157
          2       0.81      0.76      0.60      0.79      0.68      0.46      6600
          3       0.30      0.17      0.95      0.21      0.40      0.14      1156
          4       0.11      0.26      0.86      0.16      0.47      0.21       601

avg / total       0.67      0.64      0.70      0.65      0.63      0.41      9514



<Figure size 432x288 with 0 Axes>

In [ ]:
X_train['km_clust']

0        1
1        3
2        1
3        1
4        1
5        3
6        1
7        1
8        3
9        1
10       1
11       1
12       3
13       0
14       1
15       3
16       1
17       3
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       3
27       3
28       0
29       1
        ..
97186    1
97187    1
97188    1
97189    1
97190    1
97191    0
97192    3
97193    1
97194    1
97195    1
97196    0
97197    1
97198    3
97199    1
97200    1
97201    0
97202    3
97203    1
97204    1
97205    0
97206    2
97207    0
97208    3
97209    1
97210    1
97211    1
97212    1
97213    3
97214    1
97215    0
Name: km_clust, Length: 97216, dtype: int32

In [ ]:
r=X_train.copy()

In [ ]:
r

,sodium,electivesurgery,vent,dialysis,gcs,urine,wbc,temperature,respiratoryrate,heartrate,...,m12_True,m13_True,m14_True,m15_True,m16_True,m17_True,m18_True,m19_True,m20_True,km_clust
0,-0.681964,0.859496,0.0,0.0,-0.166369,-1.396133,0.337637,3.604419,0.782521,0.643651,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1
1,-0.861765,1.482495,0.0,0.0,0.176378,0.297334,0.380078,0.293250,0.850101,0.116229,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,3
2,1.059665,1.000000,1.0,0.0,0.519125,-0.543367,-0.492726,-2.126451,-1.380049,0.511796,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1
3,-0.117364,0.925698,0.0,0.0,0.519125,1.611408,-0.097963,0.165897,0.714940,0.478832,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1
4,-0.493764,0.916627,0.0,0.0,0.519125,0.443590,-0.778588,-1.871745,1.931386,1.138110,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
5,0.259036,1.098652,0.0,0.0,0.519125,1.114718,-0.438276,-0.216161,1.458324,-0.147482,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,3
6,0.552113,1.000000,0.0,0.0,0.519125,-0.369482,-0.290931,0.420602,-0.636666,0.083265,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
7,-0.493764,1.000000,0.0,0.0,0.176378,1.992667,-0.534937,-0.088808,-0.028443,-0.114519,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1
8,3.474343,0.301079,0.0,0.0,0.519125,0.189526,-0.364714,0.165897,0.241878,0.248084,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,3
9,0.447236,1.000000,0.0,0.0,0.519125,1.251597,-0.179638,0.165897,1.188002,0.709579,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1


In [ ]:
n_clusters

4

In [ ]:
X_train.shape

(97216, 83)

In [ ]:
clf.labels_

array([1, 3, 1, ..., 3, 1, 0], dtype=int32)

In [ ]:
X_train['km_clust']


0        1
1        3
2        1
3        1
4        1
5        3
6        1
7        1
8        3
9        1
10       1
11       1
12       3
13       0
14       1
15       3
16       1
17       3
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       3
27       3
28       0
29       1
        ..
97186    1
97187    1
97188    1
97189    1
97190    1
97191    0
97192    3
97193    1
97194    1
97195    1
97196    0
97197    1
97198    3
97199    1
97200    1
97201    0
97202    3
97203    1
97204    1
97205    0
97206    2
97207    0
97208    3
97209    1
97210    1
97211    1
97212    1
97213    3
97214    1
97215    0
Name: km_clust, Length: 97216, dtype: int32

In [ ]:
y_labels_train


array([1, 3, 1, ..., 3, 1, 0], dtype=int32)

In [ ]:
X_train.head()

,sodium,electivesurgery,vent,dialysis,gcs,urine,wbc,temperature,respiratoryrate,heartrate,...,m12_True,m13_True,m14_True,m15_True,m16_True,m17_True,m18_True,m19_True,m20_True,km_clust
0,-0.681964,0.859496,0.0,0.0,-0.166369,-1.396133,0.337637,3.604419,0.782521,0.643651,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1
1,-0.861765,1.482495,0.0,0.0,0.176378,0.297334,0.380078,0.293250,0.850101,0.116229,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,3
2,1.059665,1.000000,1.0,0.0,0.519125,-0.543367,-0.492726,-2.126451,-1.380049,0.511796,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1
3,-0.117364,0.925698,0.0,0.0,0.519125,1.611408,-0.097963,0.165897,0.714940,0.478832,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1
4,-0.493764,0.916627,0.0,0.0,0.519125,0.443590,-0.778588,-1.871745,1.931386,1.138110,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1


In [ ]:
X_train['km_clust'] = y_labels_train

In [ ]:
X_train['km_clust']

0        1
1        3
2        1
3        1
4        1
5        3
6        1
7        1
8        3
9        1
10       1
11       1
12       3
13       0
14       1
15       3
16       1
17       3
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       3
27       3
28       0
29       1
        ..
97186    1
97187    1
97188    1
97189    1
97190    1
97191    0
97192    3
97193    1
97194    1
97195    1
97196    0
97197    1
97198    3
97199    1
97200    1
97201    0
97202    3
97203    1
97204    1
97205    0
97206    2
97207    0
97208    3
97209    1
97210    1
97211    1
97212    1
97213    3
97214    1
97215    0
Name: km_clust, Length: 97216, dtype: int32

In [ ]:
X_train.shape

(97216, 83)

In [ ]:
y_labels_train

array([1, 3, 1, ..., 3, 1, 0], dtype=int32)

In [ ]:
X_train.head()

,sodium,electivesurgery,vent,dialysis,gcs,urine,wbc,temperature,respiratoryrate,heartrate,...,m12_True,m13_True,m14_True,m15_True,m16_True,m17_True,m18_True,m19_True,m20_True,km_clust
0,-0.681964,0.859496,0.0,0.0,-0.166369,-1.396133,0.337637,3.604419,0.782521,0.643651,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1
1,-0.861765,1.482495,0.0,0.0,0.176378,0.297334,0.380078,0.293250,0.850101,0.116229,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,3
2,1.059665,1.000000,1.0,0.0,0.519125,-0.543367,-0.492726,-2.126451,-1.380049,0.511796,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1
3,-0.117364,0.925698,0.0,0.0,0.519125,1.611408,-0.097963,0.165897,0.714940,0.478832,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1
4,-0.493764,0.916627,0.0,0.0,0.519125,0.443590,-0.778588,-1.871745,1.931386,1.138110,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
